In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import mylib as ml

# 데이터 로딩 예시 (데이터프레임 형식)
df = ml.data_000660()

# 시퀀스 길이 및 특성 수 설정
time_steps = 5  # 사용할 시퀀스 길이
features = 2     # 종가와 거래량

# 데이터 준비 함수
def prepare_data(df, time_steps):
    X = []
    y = []
    
    for i in range(len(df) - time_steps):
        X.append(df.iloc[i:i + time_steps][['close', 'volume']].values)
        # 다음 종가가 현재 종가보다 높으면 2, 같으면 1, 낮으면 0
        if df.iloc[i + time_steps]['close'] > df.iloc[i + time_steps - 1]['close']:
            y.append(2)
        elif df.iloc[i + time_steps]['close'] == df.iloc[i + time_steps - 1]['close']:
            y.append(1)
        else:
            y.append(0)

    return np.array(X), np.array(y)

# 푸리에 변환을 적용하는 함수
def apply_fourier_transform(x):
    # 입력 텐서의 크기: (batch_size, time_steps, features)
    # 각 시퀀스에 대해 푸리에 변환 적용
    fft_result = tf.signal.fft(tf.cast(tf.transpose(x, perm=[0, 2, 1]), tf.complex64))
    return tf.abs(fft_result)

# 트랜스포머 모델 정의
class TransformerBlock(layers.Layer):
    def __init__(self, embed_size, heads, dropout=0):
        super(TransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=heads, key_dim=embed_size)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward = models.Sequential([
            layers.Dense(2048, activation='relu'),
            layers.Dense(embed_size)
        ])
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)

    def call(self, x):
        attention = self.attention(x, x)
        x = self.norm1(x + self.dropout1(attention))
        feed_forward = self.feed_forward(x)
        return self.norm2(x + self.dropout2(feed_forward))

# 결합 모델 생성
def create_combined_model(input_shape):
    transformer_input = layers.Input(shape=(time_steps, features))

    # 푸리에 변환 적용
    fourier_output = layers.Lambda(apply_fourier_transform)(transformer_input)
    fourier_output = layers.Flatten()(fourier_output)

    # 트랜스포머에 연결
    transformer_block = TransformerBlock(embed_size=features, heads=4)(transformer_input)
    transformer_output = layers.GlobalAveragePooling1D()(transformer_block)

    # 두 모델의 출력 결합
    combined = layers.Concatenate()([fourier_output, transformer_output])
    combined_output = layers.Dense(3, activation='softmax')(combined)  # 3개의 클래스

    return models.Model(inputs=transformer_input, outputs=combined_output)

# 모델 생성
input_shape_cnn = (time_steps, features)
combined_model = create_combined_model(input_shape_cnn)

# 모델 컴파일
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
combined_model.summary()

# 데이터 준비
X, y = prepare_data(df, time_steps)

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=3, min_lr=1e-6)
combined_model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])


c:\big18\dl-dev\dl-dev\project\mylib.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 5, 2)]       0           []                               
                                                                                                  
 lambda_1 (Lambda)              (None, 2, 5)         0           ['input_2[0][0]']                
                                                                                                  
 transformer_block (Transformer  (None, 5, 2)        10340       ['input_2[0][0]']                
 Block)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 10)           0           ['lambda_1[0][0]']           

KeyboardInterrupt: 